In [ ]:
# Create an agent/agents that searches AQI in any given city using -
# https://python.langchain.com/docs/integrations/tools/openweathermap/

# Then creates a python code to predict the AQI after 3 days.

# You may use Langchain or CrewAI

# Expected files: Python notebook with Agent Verbose COT. Result for Atleast
# two cities.

In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools.riza.command import ExecPython
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
import traceback
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.utilities import OpenWeatherMapAPIWrapper

In [2]:
def exec_python_code(code: str) -> str:
    """Executes Python code and returns the output or error message."""
    local_vars = {}
    try:
        exec(code, {}, local_vars)  # Execute code in an isolated scope
        return str(local_vars)  # Return local variables as the result
    except Exception as e:
        return str(traceback.format_exc())  # Return error traceback if execution fails

# Define the tool
exec_python_tool = Tool(
    name="Exec Python",
    func=exec_python_code,
    description="Executes arbitrary Python code and returns the output."
)

In [ ]:
from dotenv import load_dotenv
import os
from crewai_tools import SerperDevTool
# Load environment variables from .env file
load_dotenv()
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPER_DEV_API_KEY")

serper_search = GoogleSerperAPIWrapper(serper_api_key= os.getenv("SERPER_DEV_API_KEY"))

#define serper search tool
serper_search_tool = Tool(
    name="Serper Search",
    func=serper_search.run,
    description="A tool for performing web searches using Serper.dev."
)
weather = OpenWeatherMapAPIWrapper(openweathermap_api_key =os.getenv("OPENWEATHERMAP_API_KEY"))

#define weather data search tool
weather_search_tool = Tool(
    name="Weather Data Search",
    func=weather.run,  # Use the weather API function here instead of serper_search.run
    description="A tool for retrieving weather and AQI data using OpenWeatherMap API."
)


llm_openai = ChatOpenAI(
    model_name="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Load standard tools
standard_tools = load_tools(["terminal"], llm=llm_openai,allow_dangerous_tools=True)

tools = standard_tools + [exec_python_tool, weather_search_tool, serper_search_tool]

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(tools, 
                         llm_openai, 
                         agent='zero-shot-react-description', 
                         handle_parsing_errors=True,
                        max_iterations=15,  # Set to an appropriate number
                        memory = memory,
                        verbose=True)


In [ ]:
# Correct input format as a dictionary
CITY_NAME = "Delhi"

# Run the agent with the properly formatted input
agent_query = f"""I need you to help me analyze and predict Air Quality Index (AQI) data for {CITY_NAME}. Please follow these steps:

1. Using the weather_search_tool, retrieve the current  data for {CITY_NAME} from OpenWeatherMap API. Store the AQI value in a variable.

2. Create a Python script that:
   - use the api key is stored in .env file as OPENWEATHERMAP_API_KEY
   - use the documentation at https://openweathermap.org/api/geocoding-api to get the latitude and longitude of {CITY_NAME}
   - use the documentation at https://openweathermap.org/api/air-pollution, use latitude and longitude to get the forecast AQI data for the next 3 days
   to get the forecast use https://openweathermap.org/api/air-pollution 
   - Outputs:
     * Predicted AQI values for the next 3 days

3. Execute the code using the Python execution tool and display:
   - The forecast AQI values for the next 3 days

Please provide the results in a clear, formatted output with:
- Current AQI status
- 3-day predictions with confidence intervals
"""
response = agent.invoke(agent_query)

# Output the response
print(response)



> Entering new AgentExecutor chain...
I need to break down this process into logical steps that correspond to the requirements. 

1. First, I will retrieve the current AQI data for Delhi using the Weather Data Search tool.
2. Next, I will prepare a Python script to obtain the latitude and longitude of Delhi, followed by using those coordinates to retrieve forecast AQI data for the next three days from the OpenWeatherMap API.
3. I will execute the script and then provide the required results formatted appropriately, including visualizations and health recommendations.

Let's begin by retrieving the current AQI data for Delhi.

Action: Weather Data Search  
Action Input: "Delhi"  
Observation: In Delhi, the current weather is as follows:
Detailed status: mist
Wind speed: 2.06 m/s, direction: 260°
Humidity: 94%
Temperature: 
  - Current: 14.05°C
  - High: 14.05°C
  - Low: 14.05°C
  - Feels like: 13.97°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought:The observation retrieved the gene

NotFoundError: Unable to find the resource